# 朴素贝叶斯与应用

## 1.贝叶斯理论简单回顾

在我们有一大堆样本（包含**特征**和**类别**）的时候，我们非常容易通过统计得到$p(特征|类别)$。

$$p(x)p(y|x) = p(y)p(x|y)$$

变化公式得到：
$$p(特征)p(类别|特征) = p(类别)p(特征|类别)$$$$p(类别|特征) = \frac{p(类别)p(特征|类别)}{p(特征)}$$

## 2.独立假设

<p>看起来很简单，但实际上，你的特征可能是很多维的</p>
$$p(features|class) = p({f_0, f_1, \ldots ,f_n}|c)$$<p>就算是2个维度吧，可以简单写成</p>
$$p({f_0, f_1}|c) = p(f_1|c, f_0)p(f_0|c)$$<p>这时候我们加一个特别牛逼的假设：特征之间是独立的。这样就得到了</p>
$$p({f_0, f_1}|c) = p(f_1|c)p(f_0|c)$$<p>其实也就是：</p>
$$p({f_0, f_1, \ldots, f_n}|c) = \Pi^n_i p(f_i|c)$$

## 3.贝叶斯分类器

<p>OK，回到机器学习，其实我们就是对每个类别计算一个概率$p(c_i)$，然后再计算所有特征的条件概率$p(f_j|c_i)$，那么分类的时候我们就是依据贝叶斯找一个最可能的类别：</p>
$$p(class_i|{f_0, f_1, \ldots, f_n})= \frac{p(class_i)}{p({f_0, f_1, \ldots, f_n})} \Pi^n_j p(f_j|c_i)$$

## 4. 文本分类问题

下面我们来看一个文本分类问题，经典的新闻主题分类，用朴素贝叶斯怎么做。

In [1]:
# coding:utf-8
import os
import time
import random
import jieba
import nltk
import sklearn
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt

In [2]:
def make_word_set(words_file):
    '''
    读取指定文本文件，获取词汇，汇集词汇关键字。
    '''
    words_set = set()
    with open(words_file, 'r') as fp:
        for line in fp.readlines():
            word = line.strip()
            if len(word)>0 and word not in words_set: # 去重
                words_set.add(word)
    return words_set
            

In [3]:
def text_processing(folder_path, test_size=0.2):
    '''
    文本预处理，遍历加载Database目录下的新闻语料。
    根据test_size参数，手动建立测试集和训练集文本语料。
    '''
    folder_list = os.listdir(folder_path)
    data_list = []
    class_list = []
    
    # 遍历文件夹，读取新闻正文并分词，获取对应的新闻类别标签列表
    for folder in folder_list:
        if folder == '.DS_Store':
            continue
        else:
            new_folder_path = os.path.join(folder_path, folder)
            files = os.listdir(new_folder_path)
        
            # 读取文件
            j = 1
            for file in files:
                if j > 100: # 怕内存爆掉，只取100个样本文件
                    break
                with open(os.path.join(new_folder_path, file), 'r') as fp:
                    raw = fp.read()

                # 开启jieba并行分词模式，参数为并行进程数。不支持windows操作系统
                jieba.enable_parallel(4)

                word_cut = jieba.cut(raw, cut_all=False) # 精确模式，返回的结构式一个可迭代的generator
                word_list = list(word_cut)

                # 关闭并行分词模式
                jieba.disable_parallel()

                data_list.append(word_list)  # 训练集list
                # class_list.append(folder.decode('utf-8')) # 类别Label
                class_list.append(folder)
                j += 1
    
    # 手动拆分训练集和测试集
    data_class_list = list(zip(data_list, class_list))
    random.shuffle(data_class_list) # 将序列的所有元素随机排序。
    index = int(len(data_class_list) * test_size) + 1
    train_list = data_class_list[index: ]
    test_list = data_class_list[ :index]
    train_data_list, train_class_list = zip(*train_list)
    test_data_list, test_class_list = zip(*test_list)
    
    # 或者使用sklearn自带的部分拆分训练集和测试集
    # from sklearn.model_selection import train_test_split
    # train_data_list, test_data_list, train_class_list, test_class_list = train_test_split(data_list, class_list, test_size=test_size)
    
    # 统计词频
    all_words_dict = {}
    for word_list in train_data_list:
        for word in word_list:
            if all_words_dict.get(word):
                all_words_dict[word] += 1
            else:
                all_words_dict[word] = 1
    
    # key函数利用词频进行降序排序
    all_words_tuple_list = sorted(all_words_dict.items(), key=lambda f:f[1], reverse=True)
    all_words_list = list(zip(*all_words_tuple_list))[0]
    
    return all_words_list, train_data_list, test_data_list, train_class_list, test_class_list
    

In [4]:
def words_dict(all_words_list, deleteN, stopwords_set=set()):
    '''
    处理词汇表，构建矩阵特征词汇表
    去重中止词，去除纯数字组成的字符串，词汇长度在2-4之间。
    '''
    feature_words = []
    n = 1
    for t in range(deleteN, len(all_words_list), 1):
        if n > 1000:  # feature_words的维度大小限制在1000以内
            break
        
        if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set and 1<len(all_words_list[t]) < 5:
            feature_words.append(all_words_list[t])
            n += 1
    return feature_words

In [5]:
def text_features(train_data_list, test_data_list, feature_words, flag='nltk'):
    '''
    文本特征处理，构造词袋。
    根据特征词汇feature_words，构造1000维度的1-hot向量。
    '''
    def text_features(text, feature_words):
        text_words = set(text)
        if flag == 'nltk':
            features= {word:1 if word in text_words else 0 for word in feature_words}
        elif flag == 'sklearn':
            features = [1 if word in text_words else 0 for word in feature_words]
        else:
            features = []
        return features
    
    train_feature_list = [text_features(text, feature_words) for text in train_data_list]
    test_feature_list = [text_features(text, feature_words) for text in test_data_list]
    return train_feature_list, test_feature_list

In [6]:
def text_classifier(train_feature_list, test_feature_list, train_class_list, test_class_list, flag='nltk'):
    '''
    文本分类器，返回测试集准确率。
    '''
    if flag == 'nltk':
        # 使用nltk分类器
        train_flist = zip(train_feature_list, train_class_list)
        test_flist = zip(test_feature_list, test_class_list)
        classifier = nltk.classify.NaiveBayesClassifier.train(train_flist)
        test_accuracy = nltk.classify.accuracy(classifier, test_flist)
    
    elif flag =='sklearn':
        classifier = MultinomialNB().fit(train_feature_list, train_class_list)
        test_accuracy = classifier.score(test_feature_list, test_class_list)
    
    else:
        test_accuray = []
    
    return test_accuracy

In [7]:
print('start'.center(66,'-'))

------------------------------start-------------------------------


In [ ]:
# 1. 文本预处理
folder_path = './Database/SogouC/Sample'
all_words_list, train_data_list, test_data_list, train_class_list, test_class_list = text_processing(folder_path, test_size=0.2)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache
Loading model cost 0.723 seconds.
Prefix dict has been built succesfully.


In [ ]:
# 2. 生成stopwords set
stopword_file = './files/stopwords_cn.txt'
stopwords_set = make_word_set(stopword_file)

In [ ]:
# 3. 文本特征提取和分类
# flag = 'nltk'
flag = 'sklearn'
deleteNs = range(0, 1000, 20)  # 每20个，交叉验证，选取多大的特征维度效果更佳。
test_accuracy_list = []
for deleteN in deleteNs:
    feature_words = words_dict(all_words_list, deleteN, stopwords_set)
    train_feature_list, test_feature_list = text_features(train_data_list, test_data_list, feature_words, flag)
    test_accuracy = text_classifier(train_feature_list, test_feature_list, train_class_list, test_class_list, flag)
    test_accuracy_list.append(test_accuracy)

print(test_accuracy_list)

In [ ]:
# 4. 结果评价
plt.figure()
plt.plot(deleteNs, test_accuracy_list)
plt.title('Relationship of deleteNs and test_accuracy')
plt.xlabel('deleteNs')
plt.ylabel('test_accuray')
plt.show()
